In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [15]:
# 그래프 그려서 추세 맞는지 보기
stockDF = pd.read_csv('../../data/stock.csv')
stockDF.head()

,Date,Open,High,Low,Volume,Close
0,2015-12-16,120,123,118,13181000,123
1,2015-12-17,124,126,122,17284900,123
2,2015-12-18,121,122,118,17948100,118
3,2015-12-21,120,120,116,11670000,117
4,2015-12-22,117,117,115,9689000,116


In [19]:
stockDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    967 non-null    object
 1   Open    967 non-null    int64 
 2   High    967 non-null    int64 
 3   Low     967 non-null    int64 
 4   Volume  967 non-null    int64 
 5   Close   967 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 45.5+ KB


In [ ]:
# 추세선 그래프 그리기

In [17]:
train = stockDF.sample(frac=0.8, random_state=200)
test = stockDF.drop(train.index)
val = train.sample(frac=0.1, random_state=200)

In [18]:
ytrain = train['Close']
xtrain = train.drop(['Close','Date'], axis=1)
ytest = test['Close']
xtest = test.drop(['Close','Date'], axis=1)
yval = val['Close']
xval = val.drop(['Close','Date'], axis=1)

In [9]:
class stock_dataset(Dataset):

    def __init__(self, data, label, window_size=30):
        super(stock_dataset, self).__init__()
        # 라벨/데이터 정규화
        self.data = data / np.max(self.data)
        self.label = label / np.max(self.label)
        self.window_size = window_size
        self.len = len(data) - window_size
        
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        data = self.data[index:index+self.window_size]  # 입력 데이터 30일치 읽기
        # 이전 한달 데이터를 가지고 하루의 종가 데이터 예측
        label = self.target[index+self.window_size]   # 다음날 종가 데이터
        return data,label

In [13]:
trainDS = stock_dataset(xtrain, ytrain)
testDS = stock_dataset(xtest, ytest)
valDS = stock_dataset(xval, yval)

In [14]:
trainDL = DataLoader(trainDS, batch_size=32, shuffle=True)
valDL = DataLoader(valDS, batch_size=32, shuffle=True)